In [84]:
## import data manipulation packages
import pandas as pd
import numpy as np
from sklearn.neighbors import DistanceMetric
from math import radians

## import dataset as variable 'city' and drop NaN
cities = pd.read_excel('small_cities.xlsx')
ct = cities.dropna(axis = 'rows', how = 'any')
#add london start 
London_st = ct.loc[(ct['city'] == 'London') & (ct['iso3'] == 'GBR')]
London_st['city']='London_st' 
London_st['lng'] = London_st['lng'] + 0.2
ct=ct.append(London_st)


## concatenate iso2 and city to get unique id
ct['ID'] = ct['city'].map(str) + ct['iso2'].map(str)

<ipython-input-84-22518efb9358>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  London_st['city']='London_st'
<ipython-input-84-22518efb9358>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  London_st['lng'] = London_st['lng'] + 0.2


In [85]:
## drop not usable columns
ct = ct.drop(['city_ascii', 'country', 'iso2', 'admin_name', 'capital', 'id'], axis = 1)

In [87]:
#population
pop = np.where(ct['population']< 200000 , 0 , 2 )
#same state 
zero_data = np.ones(shape=(len(ct),len(ct)))
st = pd.DataFrame(zero_data , columns = ct['iso3'], index = ct['iso3'])
for i in ct['iso3']:
    st.loc[ i , i ] = 0   
st = st * 2

In [89]:
#metto i giusti indici alle matrici 
direction = pd.DataFrame( drdf.to_numpy() , columns = ct['ID'], index = ct['ID'])
same_state = pd.DataFrame( st.to_numpy() , columns = ct['ID'], index = ct['ID'])
popolation = pd.DataFrame( pop , index = ct['ID'])





In [108]:
## coversion of coordinated in degrees to radians
ct['lat'] = np.radians(ct['lat'])
ct['lng'] = np.radians(ct['lng'])

In [109]:
## get the scipy metrics
dist = DistanceMetric.get_metric('haversine')


In [110]:
## prepare the array to get in input for the distance calculation
ct[['lat','lng']].to_numpy()

array([[ 1.08717043e-02,  4.25526775e-02],
       [-1.89307542e-03,  3.25468787e-02],
       [ 8.73033525e-03,  2.35256033e-02],
       [ 5.77758722e-03,  2.21862919e-02],
       [ 4.44613494e-03,  3.68517625e-02],
       [ 9.49391974e-03,  3.70008727e-02],
       [-7.17386208e-03, -1.42054983e-02],
       [ 1.14485279e-02,  3.86864123e-02],
       [ 5.91972170e-03, -3.01977301e-02],
       [ 7.04543538e-03,  3.45006643e-02],
       [ 1.21557581e-02,  3.54548480e-02],
       [ 9.15561163e-03,  9.51606542e-03],
       [ 6.86641172e-03,  2.69092327e-02],
       [ 1.69841879e-02,  1.14590372e-02],
       [ 4.18848952e-03,  3.06191378e-02],
       [-1.05396713e-02, -1.77841437e-02],
       [ 6.86455355e-03,  3.47428352e-02],
       [ 7.22433719e-03,  2.75362267e-02],
       [ 1.56899904e-02, -3.88387210e-05],
       [ 1.56899904e-02,  2.20847629e-05]])

In [111]:
## pass pairwise function to get the distance / multiply for 6373 as earth sperical radius is 6373kms

dist.pairwise(ct [['lat','lng']].to_numpy())*6373

array([[0.00000000e+00, 1.03362955e+02, 1.22019253e+02, 1.33788996e+02,
        5.47434229e+01, 3.64531288e+01, 3.79556670e+02, 2.49109068e+01,
        4.64694404e+02, 5.68127010e+01, 4.59657650e+01, 2.10815623e+02,
        1.02907702e+02, 2.01933245e+02, 8.71648715e+01, 4.08013367e+02,
        5.59395557e+01, 9.84784968e+01, 2.73143065e+02, 2.72757299e+02],
       [1.03362955e+02, 0.00000000e+00, 8.88201758e+01, 8.21548401e+01,
        4.88346155e+01, 7.79231119e+01, 2.99844161e+02, 9.35965329e+01,
        4.02957575e+02, 5.83100605e+01, 9.14310409e+01, 1.62789856e+02,
        6.63867390e+01, 1.80369367e+02, 4.06583418e+01, 3.25451442e+02,
        5.75401895e+01, 6.63017141e+01, 2.35965890e+02, 2.35624283e+02],
       [1.22019253e+02, 8.88201758e+01, 0.00000000e+00, 2.06630512e+01,
        8.92067288e+01, 8.60120964e+01, 2.60946814e+02, 9.81556130e+01,
        3.42837743e+02, 7.07613476e+01, 7.90932027e+01, 8.93203433e+01,
        2.46186434e+01, 9.31642815e+01, 5.36788252e+01, 2.9049

In [112]:
## create the distance matrix with cities in the indexes
distance = pd.DataFrame(dist.pairwise(ct[['lat','lng']].to_numpy())*6373, columns = ct['ID'], index = ct['ID'])


In [113]:
## identify 'London' ID 
ct.loc[(ct['city'] == 'London') & (ct['iso3'] == 'GBR')]

,city,lat,lng,iso3,population,ID
19,London,0.01569,-0.000039,GBR,10979000,LondonGB


In [114]:
## get starting line from distance matrix
distance.loc[['LondonGB']]
from pandas import DataFrame


In [115]:
#create the graph 
graph = pd.DataFrame( np.zeros(shape=(len(ct),len(ct))) , columns = ct['ID'], index = ct['ID'])

for i in distance :
    graph[i] = distance[i]*direction[i]

In [116]:
## popolare per criteri (più vicino +2, secondo più vicino +4, terzo più vicino +8, il resto 'no go')
dis = graph.replace(0, 0)
dis = dis.replace(dis.apply(lambda x: x[x > 0].min(axis=0)), 2)
dis = dis.replace(dis.apply(lambda x: x[x > 2].min(axis=0)), 4)
dis = dis.replace(dis.apply(lambda x: x[x > 4].min(axis=0)), 8)
#dis = dis.replace(dis.apply(lambda x: x[x > 8]), 'no go')

In [118]:
#parte messa solo perchè l'utima riga sopra non mi va 
dis = dis.where( (dis <= 8)  , 0  ) 

In [119]:
for i in dis :
    graph[i] = (dis[i] + same_state[i] + pop) * dis[i] / dis[i]
    
graph

#gr.T.tolist

ID,TokyoJP,JakartaID,DelhiIN,MumbaiIN,ManilaPH,ShanghaiCN,São PauloBR,SeoulKR,Mexico CityMX,GuangzhouCN,BeijingCN,CairoEG,KolkātaIN,MoscowRU,BangkokTH,Buenos AiresAR,ShenzhenCN,DhakaBD,LondonGB,London_stGB
ID,,,,,,,,,,,,,,,,,,,,
TokyoJP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
JakartaID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DelhiIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0
MumbaiIN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,6.0,6.0,NaN,NaN,NaN,12.0,12.0,8.0,8.0
ManilaPH,8.0,8.0,NaN,8.0,NaN,12.0,NaN,NaN,12.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN
ShanghaiCN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
São PauloBR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SeoulKR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mexico CityMX,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN


In [120]:
#parte messa solo perchè l'utima riga sopra non mi va 

graph = graph.where( (graph > 1 )  , 0)
gr=np.array(graph).T

In [121]:
gr_ls=gr.tolist()
print(gr_ls)

[[0.0, 0.0, 0.0, 0.0, 8.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 12.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 12.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 12.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 12.0, 0.0], [8.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 6.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.

In [106]:
# Python program for Dijkstra's single  
# source shortest path algorithm. The program is  
# for adjacency matrix representation of the graph 
  
# Library for INT_MAX 
import sys 
   
class Graph(): 
   
    def __init__(self, vertices): 
        self.V = vertices 
        self.graph = [[0 for column in range(vertices)]  
                    for row in range(vertices)] 
   
    def printSolution(self, dist): 
        print ("Vertex tDistance from Source") 
        for node in range(self.V): 
            print (node, "t", dist[node]) 
   
    # A utility function to find the vertex with  
    # minimum distance value, from the set of vertices  
    # not yet included in shortest path tree 
    def minDistance(self, dist, sptSet): 
   
        # Initilaize minimum distance for next node 
        min = sys.maxsize 
   
        # Search not nearest vertex not in the  
        # shortest path tree 
        for v in range(self.V): 
            if dist[v] < min and sptSet[v] == False: 
                min = dist[v] 
                min_idx= v
                
   
        return min_idx
   
    # Funtion that implements Dijkstra's single source  
    # shortest path algorithm for a graph represented  
    # using adjacency matrix representation 
    def dijkstra(self, src): 
   
        dist = [sys.maxsize] * self.V 
        dist[src] = 0
        sptSet = [False] * self.V 
   
        for cout in range(self.V): 
   
            # Pick the minimum distance vertex from  
            # the set of vertices not yet processed.  
            # u is always equal to src in first iteration 
            u = self.minDistance(dist, sptSet) 
   
            # Put the minimum distance vertex in the  
            # shotest path tree 
            sptSet[u] = True
   
            # Update dist value of the adjacent vertices  
            # of the picked vertex only if the current  
            # distance is greater than new distance and 
            # the vertex in not in the shotest path tree 
            for v in range(self.V): 
                if self.graph[u][v] > 0 and \
                     sptSet[v] == False and \
                     dist[v] > dist[u] + self.graph[u][v]: 
                     dist[v] = dist[u] + self.graph[u][v] 
   
        self.printSolution(dist) 
   
# Driver program 
G = Graph(9) 
G.graph =  gr_ls

  
# This code is contributed by Divyanshu Mehta 

In [107]:
G.dijkstra(0); 

UnboundLocalError: local variable 'min_idx' referenced before assignment

In [ ]:
import sys

# Function to find out which of the unvisited node 
# needs to be visited next
def to_be_visited():
  global visited_and_distance
  v = -10
  # Choosing the vertex with the minimum distance
  for index in range(number_of_vertices):
    if visited_and_distance[index][0] == 0 \
      and (v < 0 or visited_and_distance[index][1] <= \
      visited_and_distance[v][1]):
        v = index
  return v

# Creating the graph as an adjacency matrix
vertices = [[0, 1, 1, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [0, 0, 0, 0]]
edges =  [[0, 3, 4, 0],
          [0, 0, 0.5, 0],
          [0, 0, 0, 1],
          [0, 0, 0, 0]]

number_of_vertices = len(vertices[0])
